In [2]:
import numpy as np
from numpy import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn

# Generate data

In [102]:
class params:
    def __init__(self, t, dist, n=10, gen_size = 100, noise=0):
        self.t = t
        self.dist = dist
        self.n = n 
        self.gen_size = gen_size
        self.noise = noise

In [103]:
#Evaluations are representet as two sets, neg and pos where each varible *representet as an int) is in either neg or pos
class Assignment:
    def __init__(self, varibles):
        self.varibles = varibles
        
           

a1 = [1,1,0,0,0,1,1,0,1,0]   

In [104]:
#Hypothesies are representet as two sets neg and pos, where eatch varible can only bee in one set, but can be in neither
class Conjunction:
    def __init__(self, varibles):
        self.varibles = varibles
        
    
    def evaluate(self, evl):
        if len(self.varibles) != len(evl):
               return False
        
        mistakes = list(filter(lambda xy: xy[0]!=2 and xy[0]!=xy[1], zip(self.varibles, evl)))
        return len(mistakes) < 1
    
t = Conjunction([1,2,0,2,2,0,2,2,1,2]) 
t.evaluate(a1)

False

In [105]:
class make_dist():
    def __init__(self, mean, std, n=10):
        self.mean = mean
        self.std = std
        self.n = n
    
    #generates nums that represents Assignmentuations
    def normal_dist(self):
        #cap so that the binary representations does not contain more digits than n
        cap = 2**(self.n+1)-1
        minimum  = 2**self.n 
        normal_num = np.random.normal(self.mean, self.std)
        floored_abs = abs(np.math.floor(normal_num))
        num_in_range = max(minimum, min(cap, floored_abs))
        return num_in_range
    
    # converts num to useble structure
    def num_to_Assignment(self, num):
        #comvert to bin
        bin_list = [int(x) for x in bin(num)[3:]]
        
        return bin_list
    
    
    def generate_nums(self, k=1):
        return [self.normal_dist() for _ in range(k)]
    
    def generate_examples(self, k=1):
        nums = self.generate_nums(k)
        return [self.num_to_Assignment(num) for num in nums]
        
        
        

dist = make_dist(mean=2**10 *1.5, std=200, n=10)


print(dist.generate_nums(10))
S = dist.generate_examples(100)
S[:3]

[1618, 1432, 1595, 1761, 1305, 1652, 1355, 1229, 1667, 1517]


[[1, 0, 1, 0, 0, 1, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 1, 0, 1, 1, 1],
 [0, 0, 1, 0, 1, 0, 0, 0, 1, 1]]

In [106]:
params1 = params(t=t, dist=dist, n=10, gen_size=100, noise=0)

In [107]:
#return ist of tuples (example, evaluation)
def lable_examples(S, t):
    return [(e, t.evaluate(e)) for e in S]

l_S = lable_examples(S, params1.t)
l_S[0][1]

False

In [108]:
l_S[1]

([0, 0, 0, 1, 0, 1, 0, 1, 1, 1], False)

In [109]:
list(filter(lambda x: x[1],l_S))[:3]

[([1, 0, 0, 0, 1, 0, 1, 0, 1, 1], True),
 ([1, 0, 0, 0, 1, 0, 1, 0, 1, 1], True),
 ([1, 0, 0, 1, 1, 0, 0, 1, 1, 1], True)]

In [110]:
#Flips some of the evaluations
def add_noise(labeled_S, noise):
    for e in labeled_S:
        r = np.random.rand()
        if r < noise:
            #flips evaluation in tuple
            e = (e[0], not e[1])
    return labeled_S

n_S = add_noise(l_S, params1.noise)

In [111]:
#Makes labeled examples with noise (exept if noise=0).
def prepere_noisy_examples(params):
    k = params.gen_size
    dist = params.dist
    S = dist.generate_examples(k)
    labeled_S = lable_examples(S, params.t)
    noicy_S = add_noise(labeled_S, params.n)
    return noicy_S


In [112]:
# Seperate data from labels
def seperate_labels(S):
    # mapes X to a numpy array
    x = np.array([x for (x,y) in S])
    y = [y for (x,y) in S]
    return x, y
X, y = separate_labels(n_S)

In [113]:
# Transforms boolean labels to arrays coresponsing to bolean value
# True = [1,0]    
# False = [0,1]
def categorize_labels(labels):
    array_list = []
    for y in labels:
        if y:
            array_list.append([1,0])
        else:
            array_list.append([0,1])
    return np.array(array_list)
Y = categorize_labels(y)

[(yi, Yi) for yi, Yi in zip(y,Y)][:3]

[(False, array([0, 1])), (False, array([0, 1])), (False, array([0, 1]))]

In [114]:
from sklearn.model_selection import train_test_split
# Combines function so that data can be usen to train FFNN
def generate_data_for_FFNN(params):
    S = prepere_noisy_examples(params)
    X, y = seperate_labels(S)
    Y = categorize_labels(y)
    X = np.array(X)
    return train_test_split(X,Y)


# FFNN

In [115]:
params2 = params(t=t, dist=dist, n=10, gen_size=1000, noise=0)
X_train, X_test, Y_train, Y_test = generate_data_for_FFNN(params2)

In [67]:
class FFNN_params:
    def __init__(self, num_layers=2, layer_size=10, activation='relu', n=10):
        self.num_layers = num_layers
        self.layer_size = layer_size
        self.activation = activation
        self.n = n
FFNN_params1 = FFNN_params(1,10)        

In [85]:
# Uses keras functional framework
def create_FFNN_from_params(FFNN_params):
    
    inputs = keras.Input(shape=FFNN_params.n)
    x = inputs
    #Creates hidden layers
    for _ in range(FFNN_params.num_layers):
        x = layers.Dense(FFNN_params.layer_size)(x)
    
    # Creates output layer
    x = layers.Dense(2, activation=FFNN_params.activation)(x)
    
    outputs = layers.Activation('softmax')(x)
    
    # Makes model
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    #compiles model
    model.compile(
        loss=keras.losses.binary_crossentropy,
        optimizer=keras.optimizers.Adam(),
        metrics=["accuracy"],
    )
    
    return model
model1 = create_FFNN_from_params(FFNN_params1)

In [69]:
#Creates different configuration for neural networks
FFNN_params_list = []

#Corespons to NN with 1 hidden layer with 10 nodes
FFNN_params1 = FFNN_params(1,10)
FFNN_params_list.append(FFNN_params1)

FFNN_params2 = FFNN_params(5,5)
FFNN_params_list.append(FFNN_params2)

FFNN_params3 = FFNN_params(2,16,keras.activations.sigmoid)
FFNN_params_list.append(FFNN_params3)

FFNN_params4 = FFNN_params(3, 64)
FFNN_params_list.append(FFNN_params4)

FFNN_params5 = FFNN_params(20, 4)
FFNN_params_list.append(FFNN_params5)

FFNN_params6 = FFNN_params(1, 256)
FFNN_params_list.append(FFNN_params6)

In [70]:
models = [create_FFNN_from_params(FFNN_params) for FFNN_params in FFNN_params_list]

In [77]:
import time
# Returns average clasification time
def pred_time(model, X):
    n = len(X)
    start = time.time()
    model.predict(X)
    end = time.time()
    duration = end-start
    average_time = duration / n
    return average_time
pred_time(model1, X_train)

0.012206950823465982

In [78]:
# Fit models
[model.fit(x=X_train, y=Y_train) for model in models]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 750 samples
750/750 [==============================] - 1s 2ms/sample - loss: 0.7252 - acc: 0.4680
Train on 750 samples
750/750 [==============================] - 2s 2ms/sample - loss: 0.6375 - acc: 0.7020
Train on 750 samples
750/750 [==============================] - 1s 2ms/sample - loss: 0.6380 - acc: 0.7827
Train on 750 samples
750/750 [==============================] - 2s 2ms/sample - loss: 0.4116 - acc: 0.8240
Train on 750 samples
750/750 [==============================] - 3s 4ms/sample - loss: 0.3884 - acc: 0.8587
Train on 750 samples
750/750 [==============================] - 2s 2ms/sample - loss: 0.4054 - acc: 0.8627


In [82]:
# Evaluate. shows number of layers, layer size, average clasificatioin time and evaluation score
[('num_layers: '+ str(params.num_layers), 
  'size_layers: ' + str(params.layer_size), 
  'time: '+str(pred_time(model, X_train)), 
  'score: ' + str(model.evaluate(x=X_test, y=Y_test))) 
 for (params, model) in zip(FFNN_params_list,models)]

250/250 [==============================] - 0s 195us/sample - loss: 0.2075 - acc: 0.9480


[('num_layers: 1',
  'size_layers: 10',
  'time: 0.0001226669947306315',
  'score: [0.642340765953064, 0.586]'),
 ('num_layers: 5',
  'size_layers: 5',
  'time: 0.00010758781433105468',
  'score: [0.5539497175216674, 0.852]'),
 ('num_layers: 2',
  'size_layers: 16',
  'time: 8.900801340738932e-05',
  'score: [0.577681824207306, 0.936]'),
 ('num_layers: 3',
  'size_layers: 64',
  'time: 0.00010120932261149088',
  'score: [0.17904866647720336, 0.948]'),
 ('num_layers: 20',
  'size_layers: 4',
  'time: 0.00010521952311197917',
  'score: [0.23179819393157958, 0.934]'),
 ('num_layers: 1',
  'size_layers: 256',
  'time: 9.498310089111328e-05',
  'score: [0.2074899945259094, 0.948]')]

In [83]:
def num_mistakes(model, X_test, Y_test):
    model = models[2]
    m_pred = np.round(model.predict(X_test))
    mis_idxs = [i for i in range(len(m_pred)) if m_pred[i][0] != Y_test[i][0]]
    return len(mis_idxs)

# FFBNN

In [84]:
import tensorflow as tf
import larq as lq

In [89]:
# Simular to create FFNN, but it uses QuantDense layers from larq insted of Dense layers
def create_FFBNN_from_params(params):
    kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")
    
    inputs = keras.Input(shape=params.n)
    x = inputs
    for _ in range(params.num_layers):
        x = lq.layers.QuantDense(params.layer_size, **kwargs)(x)
    x = lq.layers.QuantDense(2, **kwargs)(x)
    outputs = layers.Activation('softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        loss=keras.losses.binary_crossentropy,
        optimizer=keras.optimizers.Adam(),
        metrics=["accuracy"],
    )
    
    return model
BNN_model1 = create_FFBNN_from_params(FFNN_params1)

In [88]:
# Uses same params as non binary NN so that it is easyer to compare time 
BNN_params_list = FFNN_params_list
BNN_models = [create_FFBNN_from_params(BNN_params) for BNN_params in BNN_params_list]

In [90]:
#Fit models
[model.fit(x=X_train, y=Y_train) for model in BNN_models]

Train on 750 samples
750/750 [==============================] - 2s 3ms/sample - loss: 2.2478 - acc: 0.0787
Train on 750 samples
750/750 [==============================] - 3s 4ms/sample - loss: 0.2838 - acc: 0.9213
Train on 750 samples
750/750 [==============================] - 2s 3ms/sample - loss: 0.6333 - acc: 0.8813
Train on 750 samples
750/750 [==============================] - 2s 3ms/sample - loss: 1.6459 - acc: 0.8147
Train on 750 samples
750/750 [==============================] - 6s 8ms/sample - loss: 0.7911 - acc: 0.8640
Train on 750 samples
750/750 [==============================] - 3s 4ms/sample - loss: 1.2098 - acc: 0.9213


In [92]:
# Evaluate. shows number of layers, layer size, average clasificatioin time and evaluation score
[('num_layers: '+ str(params.num_layers), 
  'size_layers: ' + str(params.layer_size), 
  'time: '+str(pred_time(model, X_train)), 
  'score: ' + str(model.evaluate(x=X_test, y=Y_test))) 
 for (params, model) in zip(BNN_params_list,BNN_models)]

250/250 [==============================] - 1s 5ms/sample - loss: 0.7975 - acc: 0.9480


[('num_layers: 1',
  'size_layers: 10',
  'time: 0.0016332708994547527',
  'score: [1.9837779483795166, 0.052]'),
 ('num_layers: 5',
  'size_layers: 5',
  'time: 0.001971943219502767',
  'score: [0.22903682374954223, 0.948]'),
 ('num_layers: 2',
  'size_layers: 16',
  'time: 0.0015727968215942382',
  'score: [0.6944991278648377, 0.052]'),
 ('num_layers: 3',
  'size_layers: 64',
  'time: 0.0015946696599324543',
  'score: [0.22972750997543334, 0.948]'),
 ('num_layers: 20',
  'size_layers: 4',
  'time: 0.003129185676574707',
  'score: [0.6748758311271668, 0.948]'),
 ('num_layers: 1',
  'size_layers: 256',
  'time: 0.0014250319798787435',
  'score: [0.7975288715362548, 0.948]')]

In [93]:
[pred_time(model, X_train) for model in models]

[0.00019371128082275392,
 0.00012126859029134115,
 0.0001041714350382487,
 0.0001249996821085612,
 0.00014876524607340496,
 0.00010856755574544271]

In [94]:
[pred_time(model, X_train) for model in BNN_models]

[0.0001335026423136393,
 0.00010396067301432292,
 0.00010416666666666667,
 0.00010416285196940104,
 0.000130647341410319,
 0.0001250012715657552]

The clasification time has no significant differance between the binary and non-binary neural netwroks. This is supprising, and I am not shure why this is the case.

# Noise

In [95]:
# Same parameters as earlyer but with 0.3 noise
noisy_params = params(t=t, dist=dist, n=10, gen_size=1000, noise=0.3)

In [96]:
nX_train, nX_test, nY_train, nY_test = generate_data_for_FFNN(noisy_params)

In [97]:
# Fit and evaluate FFNN
[model.fit(x=nX_train, y=nY_train) for model in models]

[(params.num_layers, params.layer_size, model.evaluate(x=nX_test, y=nY_test)) 
 for (params, model) in zip(FFNN_params_list,models)]

Train on 750 samples
750/750 [==============================] - 0s 258us/sample - loss: 0.6443 - acc: 0.6607
Train on 750 samples
750/750 [==============================] - 0s 252us/sample - loss: 0.4612 - acc: 0.8767
Train on 750 samples
750/750 [==============================] - 0s 233us/sample - loss: 0.5408 - acc: 0.9080
Train on 750 samples
750/750 [==============================] - 0s 360us/sample - loss: 0.1853 - acc: 0.9233
Train on 750 samples
750/750 [==============================] - 0s 251us/sample - loss: 0.2853 - acc: 0.9080
Train on 750 samples
250/250 [==============================] - 0s 284us/sample - loss: 0.2652 - acc: 0.8920


[(1, 10, [0.6156107697486878, 0.738]),
 (5, 5, [0.37689025807380677, 0.876]),
 (2, 16, [0.509109901189804, 0.892]),
 (3, 64, [0.1524250770807266, 0.916]),
 (20, 4, [0.31421304678916934, 0.892]),
 (1, 256, [0.26518691778182985, 0.892])]

In [101]:
# Fit and evaluate FFBNN
[model.fit(x=nX_train, y=nY_train) for model in BNN_models]

[(params.num_layers, params.layer_size, model.evaluate(x=nX_test, y=nY_test)) 
 for (params, model) in zip(BNN_params_list,BNN_models)]


Train on 750 samples
750/750 [==============================] - 0s 267us/sample - loss: 1.8644 - acc: 0.0853
Train on 750 samples
750/750 [==============================] - 0s 213us/sample - loss: 0.2956 - acc: 0.9147
Train on 750 samples
750/750 [==============================] - 0s 209us/sample - loss: 0.6627 - acc: 0.8027
Train on 750 samples
750/750 [==============================] - 0s 340us/sample - loss: 0.4349 - acc: 0.8747
Train on 750 samples
750/750 [==============================] - 0s 295us/sample - loss: 0.6499 - acc: 0.9147
Train on 750 samples
250/250 [==============================] - 0s 316us/sample - loss: 1.6564 - acc: 0.8920


[(1, 10, [1.8029230136871337, 0.108]),
 (5, 5, [0.3423691029548645, 0.892]),
 (2, 16, [0.8635780806541443, 0.892]),
 (3, 64, [0.3426851944923401, 0.892]),
 (20, 4, [0.6433775095939637, 0.892]),
 (1, 256, [1.6564060850143432, 0.892])]

In [99]:
[pred_time(model, X_train) for model in models]

[0.00013069661458333333,
 0.00012485122680664062,
 0.00012095419565836589,
 0.00012511889139811197,
 0.00012490367889404297,
 0.00012497774759928385]

In [100]:
[pred_time(model, X_train) for model in BNN_models]

[0.00015012105305989583,
 0.00024068991343180338,
 0.00015783532460530598,
 0.00011188189188639323,
 0.00014583079020182293,
 0.00017319424947102866]

The accuracy is worse when adding noise. It is still good given the small amount of training data. 
The FFBNN seams to do worse from short NN, getting high losses when there are few hidden layers.  